In [4]:
!pip install openai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.0/951.0 kB 14.6 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.5 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [openai]11/12 [openai]c]


In [5]:
import requests
import json
import openai
import os
from dotenv import load_dotenv
from tools import load_tools, router
from collections import defaultdict



In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [ ]:
def call_model(input_messages, response_id):
    print("interior call model, ", response_id)
    response = client.responses.create(
        model="gpt-5-mini",
        input=input_messages,
        **({"tools": tools} if response_id else {}),
        **({"previous_response_id": response_id} if response_id else {})
    )

    model_out = response.output_text
    tool_calls = []

    for tool_call in response.output:
        if tool_call.type != "function_call":
            continue

        tool_calls.append({
            "call_id": tool_call.call_id,
            "name": tool_call.name,
            "args": json.loads(tool_call.arguments)
        })    
    
    return model_out, tool_calls, response.id


def run_turn(user_text, session_id, messages_out):

    history[session_id].append({"role": "user", "content": user_text})
    print(response_ids[session_id])

    model_out, tool_calls, response_id = call_model([{"role": "user", "content": user_text}], response_ids[session_id])
    response_ids[session_id] = response_id

    while True:

        if model_out:
            messages_out.append(model_out)
            history[session_id].append({"role": "assistant", "content": model_out})

        if not tool_calls:
            return messages_out
        
        print("sending tool calls")

        tool_result_history = []

        for tool_call in tool_calls:
            tool_result = router(tool_call["name"], tool_call["args"])
            tool_result_history.append({"type": "function_call_output", "call_id": tool_call["call_id"], "output": str(tool_result)})
            history[session_id].append({"type": "function_call_output", "call_id": tool_call["call_id"], "output": str(tool_result)})

        print("sending tool result history")
        model_out, tool_calls, response_id = call_model(tool_result_history, response_id)
        response_ids[session_id] = response_id
    
